In [2]:
import requests
import bs4
import pandas as pd
import time
import re
import os
import psycopg2
from psycopg2 import sql
from datetime import datetime

# Database configuration - will use environment variables in Railway
def get_db_connection():
    """Create a connection to the PostgreSQL database"""
    try:
        conn = psycopg2.connect(
            host=os.environ.get('PGHOST', 'localhost'),
            database=os.environ.get('PGDATABASE', 'postgres'),
            user=os.environ.get('PGUSER', 'postgres'),
            password=os.environ.get('PGPASSWORD', 'postgres'),
            port=os.environ.get('PGPORT', '5432')
        )
        return conn
    except Exception as e:
        print(f"Database connection error: {e}")
        return None

def setup_database():
    """Create the necessary table if it doesn't exist"""
    conn = get_db_connection()
    if not conn:
        print("Failed to connect to database, exiting.")
        return False
    
    try:
        with conn.cursor() as cur:
            # Create articles table if it doesn't exist
            cur.execute("""
                CREATE TABLE IF NOT EXISTS articles (
                    id SERIAL PRIMARY KEY,
                    title TEXT NOT NULL,
                    url TEXT UNIQUE NOT NULL,
                    scraped_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
            """)
            conn.commit()
            print("Database setup complete.")
            return True
    except Exception as e:
        print(f"Database setup error: {e}")
        return False
    finally:
        conn.close()

def save_to_database(articles_data):
    """Save the scraped articles to the PostgreSQL database"""
    conn = get_db_connection()
    if not conn:
        return
    
    try:
        with conn.cursor() as cur:
            # Insert articles using a single query for better performance
            insert_query = sql.SQL("""
                INSERT INTO articles (title, url, scraped_at)
                VALUES (%s, %s, %s)
                ON CONFLICT (url) 
                DO UPDATE SET 
                    title = EXCLUDED.title,
                    scraped_at = EXCLUDED.scraped_at
            """)
            
            # Current timestamp for all insertions
            now = datetime.now()
            
            # Prepare data for insertion
            values = [(title, url, now) for title, url in articles_data]
            
            # Execute the query for all articles
            cur.executemany(insert_query, values)
            conn.commit()
            
            print(f"Saved {len(articles_data)} articles to database.")
    except Exception as e:
        print(f"Database insertion error: {e}")
    finally:
        conn.close()

# Set the base URL for Search Engine Land
base_url = 'https://searchengineland.com/'

# Create headers to simulate a browser (prevents being blocked)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# Function to check if a URL is a valid article URL
def is_article_url(url):
    # Skip author pages
    if '/author/' in url:
        return False
        
    # Skip category/library pages
    if '/category/' in url or '/library/' in url:
        return False
        
    # Skip tag pages
    if '/tag/' in url:
        return False
        
    # Skip about/contact/advertise pages
    if any(x in url for x in ['/about', '/contact', '/advertise', '/jobs']):
        return False
        
    # Skip archive pages
    if re.search(r'/\d{4}/\d{2}/', url):
        return False
        
    # Skip URLs with just numbers at the end (likely pagination)
    if re.search(r'/\d+/?$', url):
        return False
        
    # Accept URLs that end with a numeric ID (likely an article)
    # Example: https://searchengineland.com/google-drops-ai-while-browsing-feature-453671
    if re.search(r'-\d+/?$', url):
        return True
        
    return False

# Function to extract article headlines and links from a page
def extract_articles(url):
    # Get the page content
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"Failed to access {url}: {e}")
        return [], []
    
    # Parse the HTML
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    
    # Find all links
    links = soup.find_all('a', href=True)
    
    found_titles = []
    found_urls = []
    
    # Process each link
    for link in links:
        href = link['href']
        
        # Check if it's a Search Engine Land URL
        if 'searchengineland.com' in href:
            # Check if it matches our article URL pattern
            if is_article_url(href):
                title = link.get_text(strip=True)
                
                # Skip empty or very short titles
                if not title or len(title) < 10:
                    continue
                    
                # Only add if not already in our list
                if href not in found_urls:
                    found_titles.append(title)
                    found_urls.append(href)
    
    print(f"Found {len(found_urls)} new articles on {url}")
    return found_titles, found_urls

def scrape_search_engine_land():
    """Main function to scrape Search Engine Land"""
    # Setup database first
    if not setup_database():
        return
    
    # Get the main page first
    try:
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Failed to access {base_url}: {e}")
        return
    
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    print(f"Starting to scrape: {soup.title.text}")
    
    # Create lists to store our data
    all_article_titles = []
    all_article_urls = []
    
    # Start with the homepage
    print("Starting to scrape Search Engine Land for article headlines...")
    urls_to_visit = [base_url]
    already_visited = set()
    
    # Set limit for number of pages to scrape
    max_pages = int(os.environ.get('MAX_PAGES', 5))  # Default changed to 5 pages
    pages_scraped = 0
    
    # Scrape pages until we reach our limit
    while urls_to_visit and pages_scraped < max_pages:
        # Get the next URL
        current_url = urls_to_visit.pop(0)
        
        # Skip if already visited
        if current_url in already_visited:
            continue
            
        # Mark as visited
        already_visited.add(current_url)
        pages_scraped += 1
        
        print(f"Scraping page {pages_scraped}/{max_pages}: {current_url}")
        
        # Add delay to be respectful to the server
        time.sleep(2)
        
        # Extract articles from the page
        titles, urls = extract_articles(current_url)
        
        # Add to our master lists
        all_article_titles.extend(titles)
        all_article_urls.extend(urls)
        
        # Save to database in batches
        if len(titles) > 0:
            article_data = list(zip(titles, urls))
            save_to_database(article_data)
        
        # Get the page content to find more URLs
        try:
            response = requests.get(current_url, headers=headers)
            response.raise_for_status()
        except:
            continue
            
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        
        # Find links to category pages and pagination pages
        for link in soup.find_all('a', href=True):
            href = link['href']
            
            # Add category pages but not author or tag pages
            if ('searchengineland.com' in href and 
                (('/category/' in href) or ('/page/' in href)) and
                href not in already_visited and 
                href not in urls_to_visit):
                urls_to_visit.append(href)
                
        # Also try to find pagination links by pattern matching
        # Look for /page/2, /page/3, etc.
        if '/page/' not in current_url:  # Only do this if we're not already on a paginated page
            base_path = current_url.rstrip('/')
            for page_num in range(2, 6):  # Check pages 2 through 5
                pagination_url = f"{base_path}/page/{page_num}/"
                if pagination_url not in already_visited and pagination_url not in urls_to_visit:
                    urls_to_visit.append(pagination_url)
    
    # Create a DataFrame for reporting purposes
    articles_df = pd.DataFrame({
        'title': all_article_titles,
        'url': all_article_urls
    })
    
    # Remove duplicates
    articles_df = articles_df.drop_duplicates(subset=['url'])
    
    print(f"Completed! Scraped {pages_scraped} pages and found {len(articles_df)} unique articles.")
    
    # Print a sample of the articles found
    print("\nSample of articles found:")
    for i, (title, url) in enumerate(zip(articles_df['title'].head(5), articles_df['url'].head(5))):
        print(f"{i+1}. {title}")
        print(f"   {url}")
        print()

# Run the scraper if this is the main file
if __name__ == "__main__":
    scrape_search_engine_land()

Database setup complete.
Starting to scrape: Search Engine Land - News, Search Engine Optimization (SEO), Pay-Per-Click (PPC)
Starting to scrape Search Engine Land for article headlines...
Scraping page 1/5: https://searchengineland.com/
Found 42 new articles on https://searchengineland.com/
Saved 42 articles to database.
Scraping page 2/5: https://searchengineland.com/page/2
Found 38 new articles on https://searchengineland.com/page/2
Saved 38 articles to database.
Scraping page 3/5: https://searchengineland.com/page/3
Found 36 new articles on https://searchengineland.com/page/3
Saved 36 articles to database.
Scraping page 4/5: https://searchengineland.com/page/4
Found 38 new articles on https://searchengineland.com/page/4
Saved 38 articles to database.
Scraping page 5/5: https://searchengineland.com/page/5
Found 41 new articles on https://searchengineland.com/page/5
Saved 41 articles to database.
Completed! Scraped 5 pages and found 190 unique articles.

Sample of articles found:
1. 